# **Solving the AWS Bin Packing Problem using Google OR Tools**


For my group project, we are working on optimizing AWS's storage service S3. S3 is designed in such a way that there are different types of tiers, each with different properties and use cases, based on how often the data is needed, how long it takes to retrieve the data and more variables. 

We can also extend the problem of storing a single company's information in different tiers to look at AWS's perspective. Say Amazon themselves have a certain amount of data to be stored, and $x$ number of servers, what is the best way to store the data and to minimize the servers used? This can be solved as a Bin packing problem, with the servers being the bins. 

Firstly, we will download the packages needed to use OR Tools.

In [1]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.3 MB 19.4 MB/s 
     |████████████████████████████████| 409 kB 40.5 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.11 which is incompatible.
tensorflow-metadata 1.11.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.11 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.11 which is incompatible.


In [2]:
from ortools.linear_solver import pywraplp

## **Google OR**
Google provides two main methods of solving an optimization problem:


*   The Linear Optimization Service in Google Apps Script
*   MPSolver

For this artifact, we will be using the MPSolver through Google OR to solve the bin packing problem. Google OR provides example code for different types of optimization problems and the code used here was taken from that example and modified as needed. MPSolver can solve both linear programming and mixed integer programming problems. If we want to look at pure integer programming, we can also use the CP-SAT Solver provided by Google OR.

### **Why Google OR?**
There are many different algorithms and tools available to try and solve binpacking problems, but the biggest benefit of the Google OR algorithm is its speed as well as ability to solve large problems without taking too long. An example shown later on in this artifact will demonstrate the speed at which the MPSolver solves the problem, despite the values being fairly large.  


## **Defining the problem**

The servers have different attributes, such as where they are located, and how much they can store, but for this use case, let us assume that all the servers across the world can store upto 5000 GB of data. 

AWS has over 120 data centers across the globe, which is where the data is stored. As this is an obviously large number of data centers, to simplify it, AWS has regions which cluster these data centers. We will also be dealing with regions in this artifact, and not data centers, as there are far too many data centers to optimize this problem in a simple manner. 

Each continent or large markets have a set of regions. For this artifact, I will be looking specifically at each large market, and seeing how to optimize the regions within the market. The large markets and regions can be simplified as found below. 

| Market | Number of regions |
|--------|-------------------|
| United States | 4 |
| United States (Sensitive data) | 2 |
| Americas | 3 |
| Europe | 8 |
| Asia Pacific | 13 |
| Middle East and Africa | 4 |

We will not be looking at the United States (Sensitive data) market as there are only two servers and the bin packing problem may become obsolete with so few bins. Let us then look at the other problems for each of the regions. Let's assume that across all regions the type and amount of data stored is the same as such: 

| Data Type | Amount (GB) |
|-----------|-------------|
| Photos | 972 |
| PDFs | 1328 | 
| `.ipynb` files | 755.4 |
| `.docx` files | 1229.2 |
| `.xlsx` files | 2003.12 |

We will now solve the problem for each region with this amount of data. The code for this was the base example bin packing code provided by Google OR on their website. The function below creates the data model, with the set amount of data that we have and the number of bins. 

In [3]:
def create_data_model():
    """Create the data for the example."""
    data = {}
    weights = [972, 1328, 755.4, 1229.2, 2003.12]
    data['weights'] = weights
    data['items'] = list(range(len(weights)))
    data['regions'] = [0, 1, 2, 3]
    data['region_capacity'] = 5000
    return data

The solver below is declared using `pywraplp` which is a Python wrapper for the C++ linear solver. This example uses the SCIP backend.

In [4]:
solver = pywraplp.Solver.CreateSolver('SCIP')


Below, we will then create the data model for all of our different regions. The function creates the model with only 4 regions, so for the regions with a different number of regions, we will change the value for that as well.

In [5]:
# us = United States
# americas = Americas
# eu = Europe
# apac = Asia Pacific
# me = Middle East and Africa

data_us = create_data_model()
data_americas = create_data_model()
data_eu = create_data_model()
data_apac = create_data_model()
data_me = create_data_model()
data_americas['regions'] = [0, 1, 2]
data_eu['regions'] = [0, 1, 2, 3, 4, 5, 6, 7]
data_apac['regions'] = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


Next we will create a function that uses the Google OR tool solver to find the solution to this problem, and prints out the result. 

In [6]:

def solve(data):
# Variables
# x[i, j] = 1 if item i is packed in bin j.
  x = {}
  for i in data['items']:
      for j in data['regions']:
          x[(i, j)] = solver.IntVar(0, 1, 'x_%i_%i' % (i, j))

# y[j] = 1 if bin j is used.
  y = {}
  for j in data['regions']:
      y[j] = solver.IntVar(0, 1, 'y[%i]' % j)
  for i in data['items']:
    solver.Add(sum(x[i, j] for j in data['regions']) == 1)

# The amount packed in each bin cannot exceed its capacity.
  for j in data['regions']:
      solver.Add(
          sum(x[(i, j)] * data['weights'][i] for i in data['items']) <= y[j] *
          data['region_capacity'])
      
  solver.Minimize(solver.Sum([y[j] for j in data['regions']]))
  status = solver.Solve()
  if status == pywraplp.Solver.OPTIMAL:
    num_regions = 0.
    for j in data['regions']:
      if y[j].solution_value() == 1:
        region_items = []
        region_weight = 0
        for i in data['items']:
          if x[i, j].solution_value() > 0:
            region_items.append(i)
            region_weight += data['weights'][i]
        if region_weight > 0:
          num_regions += 1
          print('Region number', j)
          print('Items packed:', region_items)
          print('Total amount:', region_weight)
          print()
    print()
    print('Number of regions used:', num_regions)
    print('Time = ', solver.WallTime(), ' milliseconds')
  else:
      print('The problem does not have an optimal solution.')
      print(status)


### **What is the function above doing?**
Before we solve our problem, let's take a look at what the function and the commands are doing.

- `solver.IntVar` assigns the integer value to the $x$ and $y$ variables, depending on whether an item has been packed in the bin or not.
- `solver.Add` lets us add constraints to the problem.
- As we're looking at how long OR takes, we want to find the wall time of the solver as well.

In [7]:
solve(data_us)

Region number 0
Items packed: [0, 1, 2, 3]
Total amount: 4284.6

Region number 1
Items packed: [4]
Total amount: 2003.12


Number of regions used: 2.0
Time =  56  milliseconds


In [8]:
solve(data_eu)

Region number 0
Items packed: [0, 2, 3, 4]
Total amount: 4959.72

Region number 1
Items packed: [1]
Total amount: 1328


Number of regions used: 2.0
Time =  91  milliseconds


In [9]:
solve(data_apac)

Region number 0
Items packed: [0, 2, 3, 4]
Total amount: 4959.72

Region number 1
Items packed: [1]
Total amount: 1328


Number of regions used: 2.0
Time =  126  milliseconds


In [10]:
solve(data_me)

Region number 0
Items packed: [1, 2]
Total amount: 2083.4

Region number 3
Items packed: [0, 3, 4]
Total amount: 4204.32


Number of regions used: 2.0
Time =  152  milliseconds


From this, we can see how the different servers in the different regions can be used to optimize the same amount of data. We'll now see how Google OR handles this for larger amounts of data. 


Consider the same problem, but with more data:

| Data Type | Amount (GB) |
|-----------|-------------|
| Photos | 802033.4 |
| PDFs | 537302.22 | 
| `.ipynb` files | 782932.90 |
| `.docx` files | 662390.29 |
| `.xlsx` files | 1002343.223 |
| `.txt` files | 49068.71 |
| `.html` files | 120812.77 |

with a capacity of 1000000 GB for each region.

In [14]:
def create_data_model_2():
    """Create the data for the example."""
    data = {}
    weights = [802033.4, 537302.22, 782932.9, 662390.29, 1002343.223, 49068.71, 120812.77]
    data['weights'] = weights
    data['items'] = list(range(len(weights)))
    data['regions'] = [0, 1, 2, 3]
    data['region_capacity'] = 10000000
    return data

In [15]:
data_us_2 = create_data_model_2()
data_americas_2 = create_data_model_2()
data_eu_2 = create_data_model_2()
data_apac_2 = create_data_model_2()
data_me_2 = create_data_model_2()
data_americas_2['bins'] = [0, 1, 2]
data_eu_2['bins'] = [0, 1, 2, 3, 4, 5, 6, 7]
data_apac_2['bins'] = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [16]:
solve(data_us_2)

Region number 0
Items packed: [0, 1, 2, 3, 4, 5, 6]
Total amount: 3956883.513


Number of regions used: 1.0
Time =  80816  milliseconds


In [17]:
solve(data_americas_2)

Region number 0
Items packed: [0, 1, 2, 3, 4, 5, 6]
Total amount: 3956883.513


Number of regions used: 1.0
Time =  80854  milliseconds


In [18]:
solve(data_eu_2)

Region number 0
Items packed: [0, 1, 2, 3, 4, 5, 6]
Total amount: 3956883.513


Number of regions used: 1.0
Time =  81022  milliseconds


In [19]:
solve(data_apac_2)

Region number 0
Items packed: [0, 1, 2, 3, 4, 5, 6]
Total amount: 3956883.513


Number of regions used: 1.0
Time =  81197  milliseconds


In [20]:
solve(data_me_2)

Region number 1
Items packed: [0, 1, 2, 3, 4, 5, 6]
Total amount: 3956883.513


Number of regions used: 1.0
Time =  83741  milliseconds


As we can see here, Google OR is able to solve the problem for large amounts of data as well, and it's able to do it in an extremely short period of time. While this is still a relatively simple optimization problem, it would be interesting to see how OR can solve more complicated problems. 

### References

Google. (n.d.). The bin packing problem &nbsp;|&nbsp; or-tools &nbsp;|&nbsp; google developers. Google. Retrieved November 25, 2022, from https://developers.google.com/optimization/bin/bin_packing 

PYWRAPLP. pywraplp API documentation. (n.d.). Retrieved December 9, 2022, from https://google.github.io/or-tools/python/ortools/linear_solver/pywraplp.html#Solver.IntVar 